## 3_DATA_SOCIAL_TABLE
지속가능경영보고서의 표 데이터로부터 사회관련 파생변수를 추출하여 Social_Table을 만들어내는 과정입니다.

## 1. 기본설정
<b> 1 - 5번</b>은 한번만 실행해주시고, <b>6. main 함수</b>만 계속 돌려주시면서 기업정보를 엑셀화하시면 됩니다.

In [53]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
!pip install wget

## 2. table_detection 설정

In [55]:
!sudo apt update

!pip install pdf2image
!sudo apt-get install -y poppler-utils

!pip install pdfminer.six
!pip install python-dateutil

Get:1 http://packages.cloud.google.com/apt cloud-sdk-buster InRelease [6786 B]
Hit:2 http://deb.debian.org/debian buster InRelease                            
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]          
Get:4 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:5 http://packages.cloud.google.com/apt google-cloud-packages-archive-keyring-buster InRelease
Hit:6 https://packages.cloud.google.com/apt google-fast-socket InRelease       
Hit:7 https://download.docker.com/linux/debian buster InRelease                
Hit:8 http://packages.cloud.google.com/apt gcsfuse-buster InRelease            
Get:9 http://deb.debian.org/debian buster-backports InRelease [51.4 kB]m       
Get:11 https://nvidia.github.io/libnvidia-container/stable/debian10/amd64  InRelease [1484 B]
Hit:12 http://packages.cloud.google.com/apt google-compute-engine-buster-stable InRelease
Hit:10 https://packages.cloud.google.com/apt kubernetes-xenial InRelease
Hi

In [56]:
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html && rm -rf /root/.cache/pip
!pip install mmdet==2.3.0 pycocotools==2.0.2 requests

!pip install mmcv-full==1.0.5 -f https://download.openmmlab.com/mmcv/dist/cpu/torch1.6.0/index.html
!pip install pillow==6.2.1

Looking in links: https://download.pytorch.org/whl/torch_stable.html
rm: cannot remove '/root/.cache/pip': Permission denied
Looking in links: https://download.openmmlab.com/mmcv/dist/cpu/torch1.6.0/index.html


In [57]:
!pip uninstall camelot --y
!pip uninstall camelot-py --y
!pip install camelot-py[cv]

Found existing installation: camelot-py 0.10.1
Uninstalling camelot-py-0.10.1:
  Successfully uninstalled camelot-py-0.10.1
  Using cached camelot_py-0.10.1-py3-none-any.whl (40 kB)


In [58]:
!pip install wget

## 2-1. 여기서 kernel을 재시작합니다.

In [59]:
# 설치가 끝나면 커널을 재시작해야합니다.
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [8]:
# 세팅을 다시 로드
import os
import pandas as pd

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
import warnings
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.cloud import aiplatform, storage

from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    
# 지역, GCS 클라이언트, BQ 클라이언트 설정
REGION = 'asia-northeast3'
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

# bucket 이름 지정: "프로젝트 이름-bucket"
BUCKET_NAME=PROJECT_ID + "-bucket"
if not gcs.lookup_bucket(BUCKET_NAME):
    bucketDef = gcs.bucket(BUCKET_NAME)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET_NAME))

Project ID:  bigfesta2022-esg-team6-proj
<Bucket: bigfesta2022-esg-team6-proj-bucket>


In [17]:
# cascadetabnet git code를 clone하고, pretrained 가중치를 다운받습니다.
!git clone https://github.com/DevashishPrasad/CascadeTabNet.git
checkpoint_url = 'https://github.com/iiLaurens/CascadeTabNet/releases/download/v1.0.0/ICDAR.19.Track.B2.Modern.table.structure.recognition.v2.pth'

if not os.path.exists("weights/"):
    os.makedirs("weights/")
    
with open('weights/ICDAR.19.Track.B2.Modern.table.structure.recognition.v2.pth', 'wb') as f:
  f.write(requests.get(checkpoint_url).content)

In [1]:
import os
from pdf2image import convert_from_path, convert_from_bytes
import math

import cv2
import re
import os
import sys
import random
import shutil
import string
import tempfile
import warnings
from itertools import groupby
from operator import itemgetter
import pandas as pd

import numpy as np
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import (
    LAParams,
    LTAnno,
    LTChar,
    LTTextLineHorizontal,
    LTTextLineVertical,
    LTImage,
)

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import requests

from camelot import utils
import camelot
import os
import tkinter
import re
import wget
from PIL import Image

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
# 기존 gpu버전으로 실행되게끔 작성되었던 file을 cpu버전의 file로 바꿔줍니다.
conf_url = 'https://raw.githubusercontent.com/iiLaurens/CascadeTabNet/mmdet2x/Config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'

with open('CascadeTabNet/Config/cascade_mask_rcnn_hrnetv2p_w32_20e.py', 'wb') as f:
  f.write(requests.get(conf_url).content)

config_file = 'CascadeTabNet/Config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = 'weights/ICDAR.19.Track.B2.Modern.table.structure.recognition.v2.pth'
model = init_detector(config_file, checkpoint_file, device='cpu')

config = '/pdfextract/cascadeTabNet/cascade_mask_rcnn_hrnetv2p_w32_20e.py'

## 2-2. 기본 code

In [4]:
def get_dataframe(bq_table):
    
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    project_number = shell_output[0]
    bqclient = bigquery.Client(project=project_number)
    
    table = bigquery.TableReference.from_string(
        project_number+"."+bq_table
    )
    
    rows = bqclient.list_rows(
        table
    )
    
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )

    return dataframe
def to_bigquery(
    dataframe,
    table_name: str,
    bucket_name: str
):
    client = bigquery.Client()
    column_list=dataframe.columns
    query_column_list=""
    
    #blob = storage.blob.Blob.from_string(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""", client=storage.Client())
    dataframe.to_csv(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""",index=False,encoding="utf-8")
    #blob.upload_from_file(temp)
    
    for name in column_list:
        query_column_list=query_column_list+name+" STRING,"
        
    query = f"""
    LOAD DATA OVERWRITE {table_name}(
    {query_column_list}
        )
    FROM FILES (
      skip_leading_rows=1,
      allow_jagged_rows=true,
      format = 'CSV',
      encoding='utf-8',
      field_delimiter=',',
      uris = ['gs://{bucket_name}/{table_name.split(".")[-1]}.csv']);"""
    
    query_job = client.query(query)
    results = query_job.result()

## 3. pdf2image

In [5]:
### camelot의 경우에는 pdf를 pdfminer.six 패캐지로 이미지를 변환하고 표를 추출해냅니다.
### 해당 이미지 사이즈는 pdf2image 패키지로 이미지를 변환했을 때와는 다른 사이즈입니다.
### 이를 해결하기 위헤, pdf2image로 우선 pdf의 각 페이지를 image로 변환하고, 각 이미지의 사이즈를 
### pdfminer.six의 변환 사이즈에 맞게 바꿔주는 과정이 필요합니다.
### get_page_layout은 이러한 pdfminer.six의 변환 사이즈(너비 높이)를 반환해주는 코드에 해당됩니다.
def get_page_layout(
    filename,
    char_margin=1.0,
    line_margin=0.5,
    word_margin=0.1,
    detect_vertical=True,
    all_texts=True,
):
    """Returns a PDFMiner LTPage object and page dimension of a single
    page pdf. See https://euske.github.io/pdfminer/ to get definitions
    of kwargs.
    Parameters
    ----------
    filename : string
        Path to pdf file.
    char_margin : float
    line_margin : float
    word_margin : float
    detect_vertical : bool
    all_texts : bool
    Returns
    -------
    layout : object
        PDFMiner LTPage object.
    dim : tuple
        Dimension of pdf page in the form (width, height).
    """
    with open(filename, "rb") as f:
        parser = PDFParser(f)
        document = PDFDocument(parser)
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        laparams = LAParams(
            char_margin=char_margin,
            line_margin=line_margin,
            word_margin=word_margin,
            detect_vertical=detect_vertical,
            all_texts=all_texts,
        )
        rsrcmgr = PDFResourceManager()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layout = device.get_result()
            width = layout.bbox[2]
            height = layout.bbox[3]
            dim = (width, height)
            break
    return width, height

# pdf2image 패캐지를 이용하여 원하는 번호의 페이지를 이미지로 변환하여 저장해주는 함수입니다.     
def pdf_to_images(filepath,page_number=None):
  # pdf_filepath에 filepath 저장
  pdf_filepath=filepath
  page_num=page_number
    
  # image 파일이 저장될 경로
  image_filepath=os.path.join(os.path.dirname(pdf_filepath),"img/")

  if not os.path.exists(image_filepath):
    os.makedirs(image_filepath)
  # 이미지 너비, 높이 얻기
  width, height=get_page_layout(pdf_filepath)
  # pdf에서 이미지 분할하고 저장하는 코드
  if os.access(pdf_filepath, os.F_OK) == True:
    imgs = convert_from_path(pdf_filepath, fmt='jpg')
    if page_num==None:
      page_num=list(range(1,len(imgs)+1))
    for i,page in enumerate(imgs,start = 1):
      if i in page_num:
        page = page.resize((int(math.floor(width)), int(math.floor(height))), Image.LANCZOS)
        page.save(image_filepath + "image_" + str(i) + '.jpg')
    
  return image_filepath 

## 4. table detection

In [6]:
### table_deteciton 진행 후 표가 존재하는 페이지와 각 표의 bbox를 반환해주는 함수입니다.
def table_detection(image_basepath):
    image_paths = os.listdir(image_basepath)

    page_list=[]
    table_region_list=[]

    for image_path in image_paths:
      document_img = cv2.imread(image_basepath + "/" + image_path)
      table_list=[]
      try:
        # 이미지를 넣고 표들의 bbox를 반환
        result = inference_detector(model, document_img)
        #  만약 table_list 길이가 0 이상이라면,
      except Exception as e: 
        print(e)
        pass
      table_coordinates=[]
      ## extracting bordered tables
      for bordered_tables in result[0][0]:
        table_coordinates.append(bordered_tables[:4].astype(int))

      ## extracting borderless tables
      for borderless_tables in result[0][2]:
        table_coordinates.append(borderless_tables[:4].astype(int))
      page_num=int(image_path.replace("image_","").replace(".jpg",""))

      ## print tables
      for table in table_coordinates:
        page_list.append(page_num)
        x1 = table[0]
        x2 = table[2]
        y1 = table[1]
        y2 = table[3]

        table_width=document_img.shape[1]
        table_height=document_img.shape[0]

        table_region_list.append([max(0,x1-5),min(table_height,(table_height-y1)+5),min(table_width,x2+5),max(0,(table_height-y2)-5)])
        print(document_img.shape[0])
        print(document_img.shape[1])

    return page_list,table_region_list

## 5. 데이터프레임 준비과정
1. data/report_url/폴더를 생성하고 이전 파이프라인에서 생성된 social_report_src.csv를 불러옵니다.
2. 그중에서 report_url가 존재하는 기업들의 정보만 남긴 데이터프레임을 생성하고, 동일한 위치에 social_semi_final.csv를 생성합니다. 
3. 테이블을 detection한 후 정보를 담을 데이터프레임 "data/report_url/social_table.csv"를 생성합니다.

In [13]:
# 1. data/report_url/폴더를 생성하고 이전 파이프라인에서 생성된 social_report_src.csv를 불러옵니다.
# 2. 그중에서 report_url가 존재하는 기업들의 정보만 남긴 데이터프레임을 생성하고, 동일한 위치에 social_semi_final.csv를 생성합니다. 
import pandas as pd

################################
if not os.path.exists("data"):
    os.makedirs("data")

!mkdir -p "data/report_url/"
################################

social_src=get_dataframe("esg_data.social_report_src")
social_src=social_src.iloc[:,1:]
social_src["report_exist"]=(~pd.isnull(social_src["report_src"]))
social_src=social_src.loc[social_src["report_exist"]==True].reset_index(drop=True)
social_src.to_csv("data/report_url/social_semi_final.csv",index=False)

In [14]:
# 3. 테이블을 detection한 후 정보를 담을 데이터프레임 "data/report_url/social_table.csv"를 생성합니다.
social_table=social_src.drop(['stock_corp_code', 'report_src', 'rcept_no_S', 'date_S','report_exist'],axis=1)
social_table = social_table.reindex(columns = social_table.columns.tolist() + ["disabled","industry_accident",'volunteer'])
social_table.to_csv("data/report_url/social_table.csv",index=False,encoding="utf-8")

## 6. Main 함수

1) 표를 엑셀화시키고 싶은 기업의 corp_idx 설정해주셔야 합니다 ("socia_semi_final.csv" 기준 몇 번째 기업뽑을건지를 의미합니다. 0부터 셉니다.)
2) pdf 열고 표 있는 페이지의 번호 적어주세요 (뽑고 싶은 변수인 장애인 파생변수, 산업재해률, 봉사시간 표가 있는 장만 적으면 됩니다.)

```
# 표가 연속된 경우: 실제 페이지 쪽수 적으면 됩니다!
start_page=95
end_page=106
page_num=list(range(start_page,end_page+1)) # 수집 원할 시 :[95,96,97,...106]

# 필요한 표가 한정된 경우: 실제 페이지 쪽수 적으면 됩니다! 95쪽, 99쪽만 필요한 경우
page_num=[95,99]
```

3) data_/excel/에 저장된 엑셀 파일이 생성됩니다.

In [15]:
import pandas as pd
report_src=pd.read_csv("data/report_url/social_semi_final.csv") 

### data/ -> data_이름/ 으로 바꿔주기

In [25]:
# 0. data/pdf & data/img 파일 모두 정리
################################
!rm -rf "data/pdf/"
!rm -rf "data/img/"

!mkdir "data/pdf/"
!mkdir -p "data/excel/"

# 1. data/report_url에서 보고서 하나 다운
##################회사 인덱스 잘 설정############################
corp_idx=0
wget.download(report_src["report_src"][corp_idx],out="data/pdf/report_temp.pdf")

'data/pdf/report_temp.pdf'

#### 여기서 report_temp 열고, 표 있는 페이지 번호 적어주세요!

In [28]:
# 2. pdf2image
###################e다운받고 직접 표 있는 칸 눈으로 확인###################
# 표가 연속된 경우: 실제 페이지 쪽수 적으면 됩니다!
# start_page=102
# end_page=104
# page_num=list(range(start_page,end_page+1)) # 수집 원할 시 :[95,96,97,...106]

# 필요한 표가 한정된 경우: 실제 페이지 쪽수 적으면 됩니다! 95쪽, 99쪽만 필요한 경우
page_num=[102,103]

##################경로에 data/있는 경우는 모두 data_이름/으로 바꿔주기##############
image_basepath=pdf_to_images("data/pdf/report_temp.pdf",page_num)

# 3. table_detection
page_list,table_region_list=table_detection(image_basepath)

# 3-1. page_list, table_region_list을 str로!!
page_list_str= ','.join(str(s) for s in page_list)

table_region_list_str=[]
for table_region in table_region_list:
    table_region_str= ','.join(str(s) for s in table_region)
    table_region_list_str.append(table_region_str)

/opt/conda/lib/python3.7/site-packages/mmdet/apis/inference.py:109: UserWarning: We set use_torchvision=True in CPU mode.
  warnings.warn('We set use_torchvision=True in CPU mode.')
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


595
841
595
841
595
841
595
841
595
841


In [29]:
# 4. camelot + toxlsx
##################경로에 data/있는 경우는 모두 data_이름/으로 바꿔주기##############
writer=pd.ExcelWriter('data/excel/'+report_src["name"][corp_idx]+"_"+report_src["code"][corp_idx].replace("'","")+'.xlsx', engine='openpyxl')

for page_num, table_region in zip(page_list, table_region_list_str):
    try:
        ##################경로에 data/있는 경우는 모두 data_이름/으로 바꿔주기##############
        tables=camelot.read_pdf('data/pdf/report_temp.pdf',flavor='stream', pages=str(page_num),table_areas=[table_region])
        table_df=tables[0].df
        table_string=str(table_df)
        if "장애인" in table_string:
            print(table_df)
            table_df.to_excel(writer, sheet_name='장애인 파생변수_'+str(page_num))
        if (("산업" in table_string)&("재해율" in table_string)):
            print(table_df)
            table_df.to_excel(writer, sheet_name='산업재해율 파생변수_'+str(page_num))
        if (("봉사" in table_string)):
            print(table_df)
            table_df.to_excel(writer, sheet_name='봉사시간 파생변수_'+str(page_num))
    except:
        print("테이블이 영역을 벗어났습니다")
        pass
    
writer.save()

         0       1     2     3     4     5     6   \
0        구분      단위  2018  2019  2020  2021    구분   
1        전체           1.7   2.2   1.9   3.6         
2                남성     -   2.3   2.0   4.1         
3        성별                                         
4                여성     -   2.2   1.8   2.5         
5       자발적                                         
6                                              여성   
7       이직률                                         
8            30세 미만     -   3.3   3.4   5.7         
9                                            비율1)   
10     연령대별  30~50세     -   0.3   1.3   2.7         
11                                                  
12                                                  
13           50세 이상     -         1.8   2.8         
14       전체           0.5   0.3   0.3   0.3         
15            남성\n%     -   0.3   0.4   0.4         
16       성별                                         
17               여성     -   0.2   0.1   0.1   

In [47]:
############ 데이터 저장할 최종 csv 불러오고 수기로 데이터 입력후 저장 ################
##### 장애인(명), 산업재해율(%), 봉사시간(인당) #########

#############social_td_본인이름.csv로 되어있는 것 불러오시면 됩니다!################
data_df = pd.read_csv("data/report_url/social_table.csv",encoding="utf-8")

#############리스트 뒤에서 세 숫자만 변경하시면됩니다! 순서는 장애인(명), 산업재해율(%), 봉사시간(인당)#########

###### 장애인(명) // 산업재해율(%) // 봉사시간(인당) ########
############없으면 np.NaN으로 채워주세요!###############
data_df.loc[corp_idx] = [report_src["code"][corp_idx],report_src["name"][corp_idx],np.NaN,np.NaN,np.NaN]

data_df.to_csv("data/report_url/social_table.csv",index=False,encoding="utf-8")